In [312]:
%matplotlib inline

In [313]:
import numpy as np
import pandas as pd

In [314]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')[:418]

In [315]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [316]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [317]:
train['Survived'].value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [318]:
train['Child'] = float('Nan')

In [319]:
train["Age"] = train["Age"].fillna(train["Age"].median())
train.loc[train['Age'] < 18, 'Child'] = 1
train.loc[train['Age'] >= 18, 'Child'] = 0

In [320]:
train[['Age', 'Child']].head()

,Age,Child
0,22.0,0.0
1,38.0,0.0
2,26.0,0.0
3,35.0,0.0
4,35.0,0.0


In [321]:
train['Survived'][train['Child'] == 1].value_counts(normalize=True)

1    0.539823
0    0.460177
Name: Survived, dtype: float64

In [322]:
train['Survived'][train['Child'] == 0].value_counts(normalize=True)

0    0.638817
1    0.361183
Name: Survived, dtype: float64

# First Prediction 

In [323]:
test_one = test
test_one['Survived'] = 0

In [324]:
test_one['Survived'] = 0
test_one.loc[test_one['Sex'] == 'female', 'Survived'] = 1
test_one['Survived'].head()

0    0
1    1
2    0
3    0
4    1
Name: Survived, dtype: int64

In [325]:
from sklearn import tree

In [326]:
train.loc[train['Sex'] == 'male', 'Sex'] = 0
train.loc[train['Sex'] == 'female', 'Sex'] = 1

In [327]:
train['Embarked'] = train['Embarked'].fillna('S')
train.loc[train['Embarked'] == 'S', 'Embarked'] = 0
train.loc[train['Embarked'] == 'C', 'Embarked'] = 1
train.loc[train['Embarked'] == 'Q', 'Embarked'] = 2

In [328]:
train['Sex'].head()

0    0
1    1
2    1
3    1
4    0
Name: Sex, dtype: object

In [329]:
train['Embarked'].head(10)

0    0
1    1
2    0
3    0
4    0
5    2
6    0
7    0
8    0
9    1
Name: Embarked, dtype: object

In [330]:
train.values

array([[1, 0, 3, ..., nan, 0, 0.0],
       [2, 1, 1, ..., 'C85', 1, 0.0],
       [3, 1, 3, ..., nan, 0, 0.0],
       ..., 
       [889, 0, 3, ..., nan, 0, 0.0],
       [890, 1, 1, ..., 'C148', 1, 0.0],
       [891, 0, 3, ..., nan, 2, 0.0]], dtype=object)

In [331]:
target = train['Survived'].values
features_one = train[['Pclass', 'Sex', 'Age', 'Fare']].values

In [332]:
my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = my_tree_one.fit(features_one, target)

In [333]:
print(my_tree_one.feature_importances_)
print(my_tree_one.score(features_one, target))

[ 0.12901815  0.31274009  0.22690391  0.33133784]
0.977553310887


In [334]:
test['Fare'] = test['Fare'].fillna(test['Fare'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())
test.loc[test['Sex'] == 'male', 'Sex'] = 0
test.loc[test['Sex'] == 'female', 'Sex'] = 1

In [335]:
test_features = test[['Pclass', 'Sex', 'Age', 'Fare']].values

In [336]:
test_features

array([[3, 0, 34.5, 7.8292],
       [3, 1, 47.0, 7.0],
       [2, 0, 62.0, 9.6875],
       ..., 
       [3, 0, 38.5, 7.25],
       [3, 0, 27.0, 8.05],
       [3, 0, 27.0, 22.3583]], dtype=object)

In [337]:
my_prediction = my_tree_one.predict(test_features)

In [338]:
PassengerId = np.array(test['PassengerId']).astype(int)
my_solution = pd.DataFrame(my_prediction, PassengerId, columns=['Survived'])
my_solution.head()

,Survived
892,0
893,0
894,1
895,1
896,1


In [339]:
print(my_solution.shape)
print(my_solution.columns)

(418, 1)
Index(['Survived'], dtype='object')


In [340]:
my_solution.to_csv("my_solution_one.csv", index_label = ["PassengerId"])

# Second Prediction 

In [341]:
from sklearn.ensemble import RandomForestClassifier

In [342]:
features_forest = train[['Pclass', 'Age', 'Sex', 'Fare', 'SibSp', 'Parch', 'Embarked']].values

In [343]:
forest = RandomForestClassifier(max_depth=10, min_samples_split=2, n_estimators=100, random_state=1)
my_forest = forest.fit(features_forest, target)

In [344]:
print(my_forest.score(features_forest, target))

0.939393939394


# Save

In [345]:
test['Embarked'] = test['Embarked'].fillna('S')
test.loc[test['Embarked'] == 'S', 'Embarked'] = 0
test.loc[test['Embarked'] == 'C', 'Embarked'] = 1
test.loc[test['Embarked'] == 'Q', 'Embarked'] = 2

In [346]:
test_features = test[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values
pred_forest = my_forest.predict(test_features)
print(len(pred_forest))

418


In [347]:
PassengerId = np.array(test['PassengerId']).astype(int)
forest_solution = pd.DataFrame(pred_forest, PassengerId, columns=['Survived'])
forest_solution.head()

,Survived
892,0
893,0
894,0
895,0
896,0


In [348]:
forest_solution.to_csv("forest_solution.csv", index_label = ["PassengerId"])